<a href="https://colab.research.google.com/github/akbism/COVID-QA/blob/main/ModelEvaluation/6_Evaluate_checkpoints_after_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script is to evaluate all checkpoints after the a finetuning experiment is over.

In [ ]:
#### Following evaluation done on result_bkp data from boot camp
#### The data was generated from the script 

#Setting up the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/LJMU/covidqa/experiments/case_12_1_3/TPU

!pip install tqdm==4.41.1
!pip install transformers datasets
# #!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
# VERSION = "1.8.1"
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/LJMU/covidqa/experiments/case_12_1_3/TPU
     |████████████████████████████████| 2.6 MB 4.1 MB/s 
     |████████████████████████████████| 264 kB 55.6 MB/s 
     |████████████████████████████████| 895 kB 42.3 MB/s 
     |████████████████████████████████| 636 kB 57.9 MB/s 
     |████████████████████████████████| 3.3 MB 32.7 MB/s 
     |████████████████████████████████| 76 kB 5.2 MB/s 
     |████████████████████████████████| 118 kB 71.3 MB/s 
     |████████████████████████████████| 243 kB 74.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#Setting the parameters of the model evaluation


In [ ]:
import datasets, collections
import numpy as np
from transformers import Trainer, TrainingArguments
from datasets import load_from_disk, load_metric, load_dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer 
from transformers.data.metrics.squad_metrics import squad_evaluate

class Args:
    model_type = 'bert'
    # model_name = 'twmkn9/distilbert-base-uncased-squad2' #'dmis-lab/biobert-base-cased-v1.1-squad'
    tokenizer_name='roberta-base'
    output_dir = './result'
    model_dir = './result_d'
    data_dir = '/content/gdrive/My Drive/Colab Notebooks/LJMU/covidqa/biobert-pytorch/datasets/QA/BioASQ/BioASQ-678b/'
    train_file = 'BioASQ-train-factoid-6_8b-full-annotated_modified.json'
    predict_file ='BioASQ-train-factoid-7b-full-annotated_modified.json'
    covid_test = '/content/gdrive/My Drive/Colab Notebooks/LJMU/covidqa/biobert-pytorch/datasets/QA/covid_test'

    # data_dir1 = '/content/gdrive/MyDrive/Colab Notebooks/LJMU/covidqa/datasets/QA/BioASQ/BioASQ-678b'
    # train_file1 = 'BioASQ-train-factoid-7_8b-full-annotated.json'
    # predict_file1 ='BioASQ-train-factoid-6b-full-annotated.json'

    # data_dir2 = '/content/gdrive/MyDrive/Colab Notebooks/LJMU/covidqa/datasets/QA'
    # predict_file2 = 'COVID-QA-Modified.json'

    train_path = "./datasets/train/"
    dev_path = "./datasets/dev/"
    val_path = "./datasets/val/"

    save_model_path = "./qa_model/"

    max_length = 384 # The maximum length of a feature (question and context)
    per_device_train_batch_size = 16
    per_device_eval_batch_size = 16
    doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
    squad_v2 = True
    epochs= 5
    warmup_steps = 200
    save_steps = 460
    evaluation_strategy="epoch"
    weight_decay=0.01
    logging_dir='logs'
    load_best_model_at_end=True
    metric_for_best_model="f1"
    do_eval = True
    learning_rate=2e-5
    n_best_size = 20
    max_answer_length = 50
    report_to = 'wandb'
    return_overflowing_tokens=True
    return_offsets_mapping=True
    padding="max_length"
eval_args=Args()

training_args = TrainingArguments(
  output_dir='./results',
  per_device_train_batch_size = eval_args.per_device_train_batch_size,
  per_device_eval_batch_size = eval_args.per_device_eval_batch_size,
  do_eval = True)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# trainer = Trainer(
# model=model,
# args=training_args,
# # eval_dataset=val_dataset
# )

# model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(eval_args.tokenizer_name)
pad_on_right = tokenizer.padding_side == "right"

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Defining important functions

In [ ]:
import os
import datasets
from tqdm.auto import tqdm
import pandas as pd

def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    args=Args()
    # import pdb
    # pdb.set_trace()
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=args.max_length,
        stride=args.doc_stride,
        return_overflowing_tokens=args.return_overflowing_tokens,
        return_offsets_mapping=args.return_offsets_mapping,
        padding=args.padding,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


def postprocess_qa_predictions(args, examples, features, raw_predictions):
    n_best_size = args.n_best_size
    max_answer_length = args.max_answer_length
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not args.squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

def format_and_score(args, qa_df, final_predictions): 
  # import pdb
  # pdb.set_trace()
  metric = load_metric("squad_v2" if args.squad_v2 else "squad")
  if args.squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
  else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
  references = [{"id": ex["id"], "answers": ex["answers"]} for ex in qa_df]
  return metric.compute(predictions=formatted_predictions, references=references)


# def format_and_evaluate(args, qa_df, final_predictions):
#   metric = load_metric("squad_v2" if args.squad_v2 else "squad")
#   if args.squad_v2:
#     formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
#   else:
#     formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
#   references = [{"id": ex["id"], "answers": ex["answers"]} for ex in qa_df]
#   return metric.compute(predictions=formatted_predictions, references=references)



def get_eval_metrics():

  args=Args()
  print("*** Reading Data ***")
  # import pdb
  # pdb.set_trace()
  qa_data = load_from_disk(args.covid_test)
  # qa_data['train']=qa_data['train'].select(range(1_000)) 
  # qa_data['validation']=qa_data['validation'].select(range(1_000)) 
  qa_data_features = qa_data["validation"].map(
      prepare_validation_features,
      batched=True,
      remove_columns=qa_data["validation"].column_names)
  # import pdb
  # pdb.set_trace()
  #### squad v1 will give error 
  result= pd.DataFrame( columns=['exact', 'f1', 'total', 'HasAns_exact', 'HasAns_f1', 'HasAns_total','NoAns_Exact',	'NoAns_F1',	'NoAns_Total',  'best_exact', 'best_exact_thresh', 'best_f1', 'best_f1_thresh', 'model_name', 'split'])
  for checkpoint in os.listdir(eval_args.model_dir):
    # import pdb
    # pdb.set_trace()
    print("*** Reading the model ***",checkpoint)
    model_name = os.path.join(eval_args.model_dir, checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    trainer = Trainer(
    model=model,
    args=training_args)

    print("*** Prediction, postprocessing and formatting ***")
    val_raw_predictions = trainer.predict(qa_data_features)
    # import pdb
    # pdb.set_trace()
    val_final_predictions = postprocess_qa_predictions(args, qa_data["validation"], qa_data_features, val_raw_predictions.predictions)
    val_score=format_and_score(args, qa_data["validation"], val_final_predictions)  

    # if args.train_file is None:
    #   train_score = val_score
    # else:
    #   train_raw_predictions = trainer.predict(qa_data_features['train'])
    #   train_final_predictions = postprocess_qa_predictions(args,qa_data["train"], qa_data_features['train'], train_raw_predictions.predictions)
    #   train_score=format_and_score(args, qa_data["train"],  train_final_predictions)  
    
    val_score['model_name'] =checkpoint
    val_score['split'] ='validation'
    # train_score['model_name'] =checkpoint
    # train_score['split'] ='train'
    print(checkpoint)
    # import pdb
    # pdb.set_trace()
    temp=pd.DataFrame.from_dict(val_score, orient='index').T
    # temp=pd.concat([pd.DataFrame.from_dict(val_score, orient='index').T, pd.DataFrame.from_dict(train_score, orient='index').T]).reset_index()
    result=result.append(temp)
  return result

#Evaluation of the checkpoints

In [ ]:
temp1 = get_eval_metrics()

*** Reading Data ***


  0%|          | 0/2 [00:00<?, ?ba/s]

*** Reading the model *** checkpoint-100


The following columns in the test set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 43501
  Batch size = 16


*** Prediction, postprocessing and formatting ***


...

loading configuration file ./result_d/checkpoint-5600/config.json
Model config RobertaConfig {
  "_name_or_path": "deepset/roberta-base-squad2",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "name": "Roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.9.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file ./result_d/checkpoint-5600/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForQuestionAnswering.

All the

*** Prediction, postprocessing and formatting ***


Post-processing 1414 example predictions split into 43501 features.


  0%|          | 0/1414 [00:00<?, ?it/s]

checkpoint-5600


#Model Results

In [ ]:
temp1

,exact,f1,total,HasAns_exact,HasAns_f1,HasAns_total,NoAns_Exact,NoAns_F1,NoAns_Total,best_exact,best_exact_thresh,best_f1,best_f1_thresh,model_name,split
0,17.1146,23.9608,1414,17.1146,23.9608,1414,NaN,NaN,NaN,17.1146,0,23.9608,0,checkpoint-100,validation
0,23.5502,35.6833,1414,23.5502,35.6833,1414,NaN,NaN,NaN,23.5502,0,35.6833,0,checkpoint-200,validation
0,23.1966,33.6418,1414,23.1966,33.6418,1414,NaN,NaN,NaN,23.1966,0,33.6418,0,checkpoint-400,validation
0,23.5502,34.8872,1414,23.5502,34.8872,1414,NaN,NaN,NaN,23.5502,0,34.8872,0,checkpoint-800,validation
0,27.5106,41.5494,1414,27.5106,41.5494,1414,NaN,NaN,NaN,27.5106,0,41.5494,0,checkpoint-1200,validation
0,29.2786,44.2027,1414,29.2786,44.2027,1414,NaN,NaN,NaN,29.2786,0,44.2027,0,checkpoint-1600,validation
0,28.3593,45.137,1414,28.3593,45.137,1414,NaN,NaN,NaN,28.3593,0,45.137,0,checkpoint-2000,validation
0,25.4597,38.4928,1414,25.4597,38.4928,1414,NaN,NaN,NaN,25.4597,0,38.4928,0,checkpoint-2400,validation
0,27.0156,41.7824,1414,27.0156,41.7824,1414,NaN,NaN,NaN,27.0156,0,41.7824,0,checkpoint-2800,validation
0,24.0453,34.3441,1414,24.0453,34.3441,1414,NaN,NaN,NaN,24.0453,0,34.3441,0,checkpoint-3200,validation


In [ ]:
# import pickle
# with open('case_1_1_2_GPU_sara_.pkl', 'wb') as fid:
#      pickle.dump(temp1, fid)